# 03 – Using Dockyter with an API backend

This notebook demonstrates how to use Dockyter through a **remote HTTP API**
instead of talking directly to the local Docker daemon.

From the notebook user’s point of view, the interface stays the same:

- `%%docker` – run a whole cell in a single container (but via the API).
- `%docker` – configure Docker mode for `!` commands (via the API).
- `%docker_backend api <url>` – switch Dockyter to the API backend.
- `%docker_status` – inspect the current configuration.

**Prerequisites**

- `dockyter` installed in this environment (`pip install dockyter`).
- The example Dockyter API server running locally, for example:

```bash
uv run uvicorn docs.api_example.server:app --host 127.0.0.1 --port 8000
```

In [2]:
# Load the Dockyter IPython extension
%load_ext dockyter

## 1. Switch Dockyter to the API backend

By default, Dockyter uses the **local Docker daemon** backend.

Here we switch to the **API backend** pointing to the example server
running on `http://127.0.0.1:8000`.

The `%docker_backend` magic:

- `%docker_backend docker` → use local Docker daemon
- `%docker_backend api <url>` → use HTTP API at the given URL


In [5]:
%docker_backend api http://127.0.0.1:8000

Switched backend to APIBackend (http://127.0.0.1:8000).
=== Dockyter Backend Status ===
Dockyter backend type: API
Dockyter backend status: available
Dockyter backend status: API backend reachable at http://127.0.0.1:8000
Dockyer redirection for '!': off


## 2. `%%docker` cells executed via the API backend

`%%docker` still works exactly the same from the notebook:

- the cell is executed in a **single container**,
- the image and arguments are passed as in the local Docker example,
- but the actual `docker run ...` happens on the **API server side**.

Below we run a simple Ubuntu container via the API backend.

In [8]:
%%docker ubuntu:22.04
echo "Hello from the API backend inside ubuntu:22.04"
pwd
ls /

Hello from the API backend inside ubuntu:22.04
/
bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var



## 3. `%docker` + `!` commands routed through the API backend

`%docker` configures Dockyter to:

- remember the Docker image/arguments,
- and reroute all `!` shell commands through the selected backend
  (here: the API backend).

From the notebook user perspective:

- `!` still looks like a local shell command,
- but in reality it runs **inside a container on the API server host**.

In [9]:
%docker ubuntu:22.04

!echo "This was executed via Dockyter's API backend"
!uname -a

Connected

This was executed via Dockyter's API backend

Linux 4cd02e6b1adc 6.6.87.2-microsoft-standard-WSL2 #1 SMP PREEMPT_DYNAMIC Thu Jun  5 18:30:46 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux



## 4. Notes on security and deployment

In this example, the API backend is:

- running on `127.0.0.1:8000`,
- started manually with `uvicorn docs.api_example.server:app ...`,
- and is intended for **local / trusted** environments only.

A real-world deployment should:

- put the API behind authentication and transport security (HTTPS),
- restrict which images/arguments can be used,
- and run on a hardened host.

From Dockyter’s point of view, the only requirement is that the API:

- exposes `GET /health` returning a 2xx status when available,
- exposes `POST /execute` with a JSON body:

  ```json
  {
    "cmd": "echo hello",
    "args": "ubuntu:22.04 -v /tmp:/tmp"
  }
```

and returns JSON of the form:

```json
{
  "stdout": "hello",
  "stderr": ""
}
```

Everything else (auth, routing, logging, etc.) is up to the API implementation.